In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

c:\users\phs\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [6]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\phs\textmining\python\text-mining-camp\note\arkwith\Contextual_Chatbots\model.tflearn


In [7]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [8]:
classify('is your shop open today?')

[('opentoday', 0.7144049)]

In [9]:
response('is your shop open today?')

We're open every day from 9am-9pm


In [10]:
classify('what kind of mopeds do you rent?')

[('mopeds', 0.9550577)]

In [11]:
response('Do you accept AMEX?')

We accept VISA, Mastercard and AMEX


In [12]:
response('what kind of mopeds do you rent?')

We have Piaggio, Vespa and Yamaha mopeds


In [13]:
response('Goodbye, see you later')

See you later, thanks for visiting


In [14]:
context

{}

In [21]:
response('we want to rent a moped')

Are you looking to rent today or later this week?


In [22]:
# show context
context

{'123': 'rentalday'}

In [17]:
response('today')

For rentals today please call 1-800-MYMOPED


In [18]:
classify('today')

[('today', 0.662425)]

In [23]:
# clear context
response("How are you", show_details=True)

context: 
tag: greeting
Hi there, how can I help?


In [24]:
response('today')
classify('today')

[('today', 0.662425)]

In [21]:
response("thanks, your great")

Any time!


In [23]:
response('today', show_details=True)

In [24]:
# show context
context

{'123': ''}

In [25]:
response('we want to rent a moped')

Are you looking to rent today or later this week?


In [26]:
# show context
context

{'123': 'rentalday'}

In [27]:
response('today', show_details=True)

tag: today
Same-day rentals please call 1-800-MYMOPED


In [28]:
response('today')
classify('today')

Same-day rentals please call 1-800-MYMOPED


[('today', 0.662425)]

In [29]:
response("thanks, your great")

My pleasure


In [30]:
response('today')
classify('today')

For rentals today please call 1-800-MYMOPED


[('today', 0.662425)]

In [31]:
response('is your shop open today?')

We're open every day from 9am-9pm


In [32]:
# show context
context

{'123': 'rentalday'}

In [33]:
classify('is your shop open today?')

[('opentoday', 0.7144049)]